In [21]:
import woodwork as ww

data = ww.demo.load_retail(nrows=100, return_dataframe=True)
data.head(5)

,order_product_id,order_id,product_id,description,quantity,order_date,unit_price,customer_name,country,total,cancelled
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.2075,Andrea Brown,United Kingdom,25.245,False
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,5.5935,Andrea Brown,United Kingdom,33.561,False
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,4.5375,Andrea Brown,United Kingdom,36.300,False
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,5.5935,Andrea Brown,United Kingdom,33.561,False
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,5.5935,Andrea Brown,United Kingdom,33.561,False


As we can see, this is a dataframe containing several different data types, including dates, categorical values, numeric values and natural language descriptions. Let's use Woodwork to create a DataTable from this data.

## Creating a DataTable
Creating a Woodwork DataTable is as simple as passing in a dataframe with the data of interest during initialization. An optional name parameter can be specified to label the DataTable.

In [22]:
dt = ww.DataTable(data, name="retail")
dt

Data Column,order_product_id,order_id,product_id,description,quantity,order_date,unit_price,customer_name,country,total,cancelled
Physical Type,Int64,Int64,category,string,Int64,datetime64[ns],float64,string,string,float64,boolean
Logical Type,Integer,Integer,Categorical,NaturalLanguage,Integer,Datetime,Double,NaturalLanguage,NaturalLanguage,Double,Boolean
Semantic Tag(s),['numeric'],['numeric'],['category'],[],['numeric'],[],['numeric'],[],[],['numeric'],[]
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.2075,Andrea Brown,United Kingdom,25.245,False
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,5.5935,Andrea Brown,United Kingdom,33.561,False
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,4.5375,Andrea Brown,United Kingdom,36.300,False
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,5.5935,Andrea Brown,United Kingdom,33.561,False
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,5.5935,Andrea Brown,United Kingdom,33.561,False


Using just this simple call, Woodwork was able to infer the logical types present in our data by analyzing the dataframe dtypes as well as the information contained in the columns. In addition, Woodwork also added semantic tags to some of the columns based on the logical types that were inferred.

We can also view the typing information with the first few columns of data with the following:

In [23]:
dt.head()

AttributeError: 'DataTable' object has no attribute 'head'

## Updating Logical Types
If the initial inference was not to our liking, the logical type can be changed to a more appropriate value. Let's change some of the columns to a different logical type to illustrate this process. Below we will set the logical type for the ``quantity``, ``customer_name`` and ``country`` columns to be ``Categorical``.

In [ ]:
dt = dt.set_types(logical_types={
    'quantity': 'Categorical',
    'customer_name': 'Categorical',
    'country': 'Categorical'
})
dt

If we now inspect the information in the `types` output, we can see that the Logical type for the three columns has been updated with the `Categorical` logical type we specified.

## Selecting Columns

Now that we have logical types we are happy with, we can select a subset of the columns based on their logical types. Let's select only the columns that have a logical type of ``Integer`` or ``Double``:

In [ ]:
numeric_dt = dt.select(['Integer', 'Double'])
numeric_dt

This selection process has returned a new ``DataTable`` containing only the columns that match the logical types we specified. After we have selected the columns we want, we can also access a dataframe containing just those columns if we need it for additional analysis.

In [ ]:
numeric_dt.to_dataframe()

## Adding Semantic Tags

Next, let's add semantic tags to some of the columns. We will add the tag of ``product_details`` to the ``description`` column and tag the ``total`` column with ``currency``.

In [ ]:
dt = dt.set_types(semantic_tags={'description':'product_details', 'total': 'currency'})
dt

We can also select columns based on a semantic tag. Perhaps we want to only select the columns tagged with ``category``:

In [ ]:
category_dt = dt.select('category')
category_dt

We can also select columns using mutiple semantic tags, or even a mixture of semantic tags and logical types:


In [ ]:
category_numeric_dt = dt.select(['numeric', 'category'])
category_numeric_dt

In [ ]:
mixed_dt = dt.select(['Boolean', 'product_details'])
mixed_dt

If we wanted to select an individual column, we just need to specify the column name. We can then get access to the data in the DataColumn using the ``to_series`` method:

In [ ]:
dc = dt['total']
dc

In [ ]:
dc.to_series()

You can also access multiple columns by supplying a list of column names:

In [ ]:
multiple_cols_dt = dt[['product_id', 'total', 'unit_price']]
multiple_cols_dt

## Removing Semantic Tags
We can also remove specific semantic tags from a column if they are no longer needed. Let's remove the ``product_details`` tag from the ``description`` column:

In [ ]:
dt = dt.remove_semantic_tags({'description':'product_details'})
dt

Notice how the ``product_details`` tag has now been removed from the ``description`` column. If we wanted to remove all user-added semantic tags from all columns, we can also do that:

In [ ]:
dt = dt.reset_semantic_tags()
dt

## Set Index and Time Index
At any point, we can designate certain columns as the DataTable's `index` and  with the methods [set_index](generated/woodwork.datatable.DataTable.set_index.rst) and [set_time_index](generated/woodwork.datatable.DataTable.set_time_index.rst). These methods can be used to assign these columns for the first time or to change the column being used as the index or time index.

Index and time index columns contain `index` and `time_index` semantic tags, respectively.

In [ ]:
dt = dt.set_index('order_product_id')
dt.index

In [ ]:
dt = dt.set_time_index('order_date')
dt.time_index

In [ ]:
dt

## List Logical Types
We can also retrieve all the Logical Types present in Woodwork. These can be useful for understanding the Logical Types, and how they will be interpreted. 

In [ ]:
from woodwork.utils import list_logical_types

list_logical_types()